In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import model_selection,naive_bayes,svm
import plotly
import plotly.graph_objs as go
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
import seaborn as sns
import shap
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean, cosine
import re
from utils import *
from sklearn.preprocessing import LabelEncoder
from math import log
from sklearn.svm import SVC
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import roc_auc_score
import shap
from imblearn.metrics import geometric_mean_score
from scipy.spatial import distance


Using TensorFlow backend.


In [2]:
train=pd.read_csv("dataset/Founta_edit/train.csv")
test=pd.read_csv("dataset/Founta_edit/test.csv")
unlabel=pd.read_csv("dataset/Founta_edit/unlabel.csv")
gold_standard=pd.read_csv("dataset/Founta_edit/gold_standard.csv")

In [3]:
print(len(gold_standard))
print(len(gold_standard[gold_standard['label']==1]))

7117
521


In [4]:
#parameter setting
C = 1
max_iter = 1000
Degree=3
Gamma='auto'

In [5]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

In [6]:
print(len(train))
print(len(test))
print(len(unlabel))

2373
2373
7117


In [7]:
train.dropna(inplace=True)
test.dropna(inplace=True)
unlabel.dropna(inplace=True)


In [8]:
#first training on train data
X_train=train['text']
y_train=train['label']
X_test=test['text']
y_test=test['label']

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train['text'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)
SVM = svm.SVC(C=1, kernel='linear', gamma='auto',probability=True)
SVM.fit(Train_X_Tfidf,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
score=geometric_mean_score(y_test, predictions_SVM)
print(score)

0.4773408230832845


In [1327]:
print(len(train))
print(len(test))
print(len(unlabel))

2839
2371
6632


In [1328]:
#training a model and predict unlabel
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train['text'])
x_train = Tfidf_vect.transform(train['text']).toarray()
x_test = Tfidf_vect.transform(test['text']).toarray()
x_pool=Tfidf_vect.transform(unlabel['text']).toarray()
y_train=train['label']
print("model prediction and predict on unlabel")
model = SVC( max_iter=max_iter, C=C, kernel='linear',probability=True,gamma=Gamma,degree=Degree)
model.fit(x_train, y_train)
model.score(x_train,y_train)
prediction=model.predict(x_pool)
probs=model.predict_proba(x_pool)
entropy=list()
for p in probs:
    ent=0
    ent1=-p[0] * log(p[0],2)
    ent2=-p[1] * log(p[1],2)
    ent=ent1+ent2
    entropy.append(ent)
df=pd.DataFrame({'index':unlabel['index'],"text":unlabel['text'],"confidence":entropy,'prediction':prediction})
df=df.sort_values(by="confidence",ascending=False)
#unlabel_length=len(unlabel)*0.1
#taking the whole unlabel space
unlabel_length=len(unlabel)
low_conf=df[0:round(unlabel_length)]

model prediction and predict on unlabel


In [1329]:
len(low_conf)

6632

In [1330]:
#calculate shapley value on low conf data
x_pool=Tfidf_vect.transform(low_conf['text']).toarray()
print("calculate Shapley value ")
explainer = shap.LinearExplainer(model, x_train,feature_dependence="independent")
shap_values_train = explainer.shap_values(x_train)
shap_values_pool = explainer.shap_values(x_pool)
n_clusters=20

print("clusterint ")
kmeans = KMeans(n_clusters= n_clusters, n_jobs=-1, max_iter=600)
kmeans.fit(shap_values_pool)

#calculate the center
similarity_to_center = list()
similarity=0
for i, instance in enumerate(shap_values_pool):
    cluster_label = kmeans.labels_[i] # cluster of this instance
    centroid = kmeans.cluster_centers_[cluster_label] # cluster center of the cluster of that instance
    #similarity = 1-cosine(instance, centroid) # 1- cosine distance gives similarity
    similarity = (1)-(distance.cosine([instance], [centroid]))
    similarity_to_center.append(similarity)

calculate Shapley value 
clusterint 


In [1331]:
cluster_labels=pd.DataFrame({"index":low_conf['index'],"text":low_conf['text'],'cluster':kmeans.labels_,
                            'uncertainty':low_conf['confidence'],'similarity':similarity_to_center,
                            'prediction':low_conf['prediction']})

In [1332]:
cluster_dict=dict()
n_cluster=20
#creating the dataframe for each cluster
for item in range(0,n_cluster):
    cluster_dict['cluster_{0}'.format(item)]=cluster_labels[cluster_labels['cluster']==item] 
#Create the dataframe for each cluster
for i in range(0,n_cluster):
    globals()['cluster_{}'.format(i)] = pd.DataFrame(cluster_dict['cluster_{}'.format(i)])

In [1333]:
for i in range(0,20):
    globals()['cluster_{}'.format(i)]=globals()['cluster_{}'.format(i)].sort_values(by="similarity",ascending=False)

In [1334]:
for i in range(0,20):
    print(len(globals()['cluster_{}'.format(i)]))

30
33
163
74
5297
51
96
44
51
19
206
91
124
60
40
3
36
178
7
29


In [1486]:
clus_name=globals()['cluster_19']
print("#samples",len(clus_name))
print("max sim",clus_name['similarity'].max())
print("min sim",clus_name['similarity'].min())
print("range ",(clus_name['similarity'].max()-clus_name['similarity'].min()))

#print("========Uncertainty==============")
#print("max unc",clus_name['uncertainty'].max())
#print("min sim",clus_name['uncertainty'].min())
#print("range ",(clus_name['uncertainty'].max()-clus_name['uncertainty'].min()))

#samples 29
max sim 0.9320539684480873
min sim 0.45648320814581944
range  0.47557076030226786


In [1487]:
#get the label for the cluster
true_label=pd.read_csv("dataset/Founta_edit/gold_standard.csv")
#get the label
new_train=true_label.merge(clus_name,on='index',how='left')
new_train=new_train.dropna()

In [1488]:
new_train.drop(['text_y'],axis=1,inplace=True)
new_train.rename(columns={"text_x": "text"},inplace=True)

In [1489]:
new_train=new_train.sort_values(by='similarity',ascending=False)

In [1490]:
l_minority=len(new_train[new_train['label']==1])
l_majority=len(new_train[new_train['label']==0])
total_samples=len(new_train)
print("majority class found {}".format(l_majority))
print("minority class found {}".format(l_minority))

#percentage of minority
print("percentage {}".format(l_minority/total_samples))
center_label=new_train['label'].head(1)
print(center_label)

new_train=new_train[['index','label','prediction','text','cluster','uncertainty','similarity']]

majority class found 29
minority class found 0
percentage 0.0
4844    0
Name: label, dtype: int64


In [1491]:
ground_truth=new_train['label']
pred=new_train['prediction']

In [1492]:
new_train.to_csv("cluster_exploration/iteration_8/cluster_19.csv")

In [1493]:
print(accuracy_score(ground_truth,pred))

1.0


In [1469]:
new_train

,index,label,prediction,text,cluster,uncertainty,similarity
3303,25776,0,0.0,let talk ways wic helps mothers young children...,16.0,0.304664,0.955785
1400,74269,0,0.0,barry manilow surgery well ordered talk sing b...,16.0,0.433939,0.953358
427,57842,0,0.0,iota omega hosts book talk community black pai...,16.0,0.358882,0.949680
2294,35595,0,0.0,enjoyable informative chat inspiring tedx talk...,16.0,0.750224,0.934344
7033,71031,0,0.0,chance go exercise awhile nice bath talk someo...,16.0,0.535303,0.934009
6956,63917,0,0.0,talk rest world proudly thank sir forgetting,16.0,0.606632,0.928588
3535,42237,0,0.0,care spoken text bebecause need advice friend ...,16.0,0.395130,0.904956
4304,15747,0,0.0,starting pm tomorrow lunchtime talk focus stub...,16.0,0.224994,0.903342
2957,41751,0,0.0,giving video benefit wouldoubt talk proof evid...,16.0,0.213173,0.897528
571,41660,0,0.0,idea asking talk somebody india talk someone,16.0,0.386819,0.894509


In [1494]:
cluster_exclude=list()
cluster_exclude=[1,4,8,15,18]

In [1495]:
#label propagation
#read the ground truth
search_idx=list()
gold_standard=pd.read_csv("dataset/Founta_edit/gold_standard.csv")
for i in range(0,20):
    if i in cluster_exclude:
        continue
    else:
        #search for the label in the ground truth
        search_index=globals()['cluster_{}'.format(i)]['index'].head(1).values[0]
        labeled_selection=gold_standard.loc[gold_standard['index']==search_index]
        labels=labeled_selection['label'].values[0]
        globals()['cluster_{}'.format(i)]['label']=labels

In [1496]:
#get the data and balance it before put back in the training dataset
print("Balancing ")
conf_data=pd.DataFrame()
for i in range(0,20):
    if i in cluster_exclude:
        continue
    else:
        conf_data=conf_data.append(globals()['cluster_{}'.format(i)])
print(len(conf_data[conf_data['label']==0]))
print(len(conf_data[conf_data['label']==1]))
majority=conf_data[conf_data['label']==0]
n_minority_class=conf_data[conf_data['label']==1]
if len(n_minority_class) > len(majority):
    cluster_size=len(majority)
elif len(n_minority_class)==0:
    cluster_size=1
else:
    cluster_size=len(n_minority_class)

Balancing 
1241
0


In [1497]:
conf_data[conf_data['label']==1]

,index,text,cluster,uncertainty,similarity,prediction,label


In [1498]:
#get the majority and cluster it based on number of minority class
embeddings = Tfidf_vect.transform(majority['text'])
#initialize the kmeans
kmeans = KMeans(n_clusters=cluster_size, random_state=0,max_iter=600,n_init=10)
kmeans.fit(embeddings)
labels = (kmeans.labels_)
#create the cluster for each low conf data
cluster_labels = pd.DataFrame({'index':majority['index'],'cluster':labels,'text':majority['text'],
                               'label':majority['label']})
cluster_dict=dict()
#creating the dataframe for each cluster
for item in range(0,cluster_size):
    cluster_dict['cluster_{0}'.format(item)]=cluster_labels[cluster_labels['cluster']==item] 
for i in range(0,cluster_size):
        globals()['cluster_{}'.format(i)] = pd.DataFrame(cluster_dict['cluster_{}'.format(i)])
center=kmeans.cluster_centers_
#print(center.shape)
#create the data frame of the cluster center
centerpd=pd.DataFrame(center)
#inverse transform the center to text
inver=Tfidf_vect.inverse_transform(centerpd)
#get the inverse word and create as a sentence
sentence=[' '.join(item) for item in inver]
#Calculate the data that close to the center for each cluster
for i in range(0,cluster_size):
    sent=list()
    cossim=list()
    for item in globals()['cluster_{}'.format(i)]['text']:
        sent.append(item)
    for k in sent:
        vector1=text_to_vector(k)
        vector2=text_to_vector(sentence[i])
        cosine=get_cosine(vector1,vector2)
        cossim.append(cosine)
    globals()['cluster_{}'.format(i)]['similarity']=cossim
    globals()['cluster_{}'.format(i)]=globals()['cluster_{}'.format(i)].sort_values(by='similarity',ascending=False)
center_data=pd.DataFrame()
for i in range(0,cluster_size):
    center_data=center_data.append(globals()['cluster_{}'.format(i)].head(1))

In [1499]:
print(len(train))
print(len(test))
print(len(unlabel))

2839
2371
6632


In [1500]:
#adding to training data
train_1=n_minority_class
train_0=center_data
new_train=pd.DataFrame()
#adding the balanced set for training
frames=[train_1,train_0] 

new_train=pd.concat(frames)
new_train.drop(['cluster','similarity'],axis=1,inplace=True)
training_data=pd.DataFrame({'index':new_train['index'],
                           'label':new_train['label'],'text':new_train['text']})

In [1501]:
print("minorty class {}".format(len(new_train[new_train['label']==1])))
print("majority class {}".format(len(new_train[new_train['label']==0])))

minorty class 0
majority class 1


In [1502]:
#add to training data and remove from unlabel pool
minority_sum=len(new_train[new_train['label']==1])
if minority_sum >= 0:
    train=train.append(training_data)
    train.to_csv("training/training.csv",index=False)

    test.to_csv("training/test.csv",index=False)

    unlabel=unlabel[~unlabel['index'].isin(train['index'])]

In [1503]:
print(len(train))
print(len(test))

2840
2371


In [1504]:
#evaluating performance
train1=pd.read_csv("training/training.csv")
test1=pd.read_csv("training/test.csv")

train1.dropna(inplace=True)
test1.dropna(inplace=True)


X_train=train1['text']
y_train=train1['label']
X_test=test1['text']
y_test=test1['label']

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train['text'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)
SVM = svm.SVC(C=1, kernel='linear', gamma='auto',probability=True)
SVM.fit(Train_X_Tfidf,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
score=geometric_mean_score(y_test, predictions_SVM)
#gmean.append(score)

In [1505]:
print(score)

0.5892165596713536
